In [1]:
# 인구밀도, 소득수준, 학원, 교통량 , 유동인구 바탕으로 커피전문점 학습 후 예측
# 동별 인구밀도(15~19년도 동별)
# 15~19년도 자료 병합 (15~18년도 자료로 학습 -> 이를 바탕으로 19년도 자료로 테스트)

# 사업체수, 사업체종사자수
# https://data.seoul.go.kr/dataList/10599/S/2/datasetView.do
library(stringr)

bw <- read.table("../Data_SemiProject/2019_business_worker.txt",
           fileEncoding ="utf-8", sep="\t", header=T)

head(bw)

기간,자치구,동,합계,합계.1,합계.2,농업.임업.및.어업,농업.임업.및.어업.1,광업,광업.1,...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정.1,교육.서비스업,교육.서비스업.1,보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업.1,예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업.1,협회.및.단체.수리.및.기타.개인.서비스업,협회.및.단체.수리.및.기타.개인.서비스업.1
기간,자치구,동,사업체수,총종사자수,여성종사자수,계,여성종사자수,계,여성종사자수,...,계,여성종사자수,계,여성종사자수,계,여성종사자수,계,여성종사자수,계,여성종사자수
2019,합계,합계,"823,624","5,226,997","2,341,383",426,77,43,7,...,"141,582","53,867","351,320","214,507","400,143","316,091","92,752","43,487","189,557","96,645"
2019,종로구,소계,"39,679","260,446","114,054",18,6,10,3,...,"15,109","5,135","13,474","7,272","17,366","12,698","6,034","3,006","7,649","3,545"
2019,종로구,사직동,"3,574","47,522","22,368",12,4,2,1,...,"6,088","2,369",786,474,"1,096",838,"1,653",794,906,475
2019,종로구,삼청동,732,"4,601","2,330",-,-,-,-,...,"1,291",335,297,158,30,20,185,128,194,121
2019,종로구,부암동,599,"3,454","1,690",-,-,1,-,...,60,13,"1,190",558,183,154,49,29,325,149


In [2]:

bw_02 <- bw
head(bw_02)

# factor -> character 변경
for(i in 4:ncol(bw_02)){
    
    bw_02[,i] <- as.character(bw_02[,i])
}
str(bw_02)

기간,자치구,동,합계,합계.1,합계.2,농업.임업.및.어업,농업.임업.및.어업.1,광업,광업.1,...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정.1,교육.서비스업,교육.서비스업.1,보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업.1,예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업.1,협회.및.단체.수리.및.기타.개인.서비스업,협회.및.단체.수리.및.기타.개인.서비스업.1
기간,자치구,동,사업체수,총종사자수,여성종사자수,계,여성종사자수,계,여성종사자수,...,계,여성종사자수,계,여성종사자수,계,여성종사자수,계,여성종사자수,계,여성종사자수
2019,합계,합계,"823,624","5,226,997","2,341,383",426,77,43,7,...,"141,582","53,867","351,320","214,507","400,143","316,091","92,752","43,487","189,557","96,645"
2019,종로구,소계,"39,679","260,446","114,054",18,6,10,3,...,"15,109","5,135","13,474","7,272","17,366","12,698","6,034","3,006","7,649","3,545"
2019,종로구,사직동,"3,574","47,522","22,368",12,4,2,1,...,"6,088","2,369",786,474,"1,096",838,"1,653",794,906,475
2019,종로구,삼청동,732,"4,601","2,330",-,-,-,-,...,"1,291",335,297,158,30,20,185,128,194,121
2019,종로구,부암동,599,"3,454","1,690",-,-,1,-,...,60,13,"1,190",558,183,154,49,29,325,149


'data.frame':	451 obs. of  44 variables:
 $ 기간                                      : Factor w/ 2 levels "2019","기간": 2 1 1 1 1 1 1 1 1 1 ...
 $ 자치구                                    : Factor w/ 27 levels "강남구","강동구",..: 23 27 24 24 24 24 24 24 24 24 ...
 $ 동                                        : Factor w/ 426 levels "가락1동","가락2동",..: 89 400 211 163 171 151 392 118 31 9 ...
 $ 합계                                      : chr  "사업체수" "823,624" "39,679" "3,574" ...
 $ 합계.1                                    : chr  "총종사자수" "5,226,997" "260,446" "47,522" ...
 $ 합계.2                                    : chr  "여성종사자수" "2,341,383" "114,054" "22,368" ...
 $ 농업.임업.및.어업                         : chr  "계" "426" "18" "12" ...
 $ 농업.임업.및.어업.1                       : chr  "여성종사자수" "77" "6" "4" ...
 $ 광업                                      : chr  "계" "43" "10" "2" ...
 $ 광업.1                                    : chr  "여성종사자수" "7" "3" "1" ...
 $ 제조업                                    : chr  "계" "265

In [3]:
# 동별 사업체, 총 종사자 수 , 업종별 종사자 수 데이터
# 업종별 컬럼 앞에 .1 -> 해당 업종 여성 종사자 수


bw_02 <- subset(bw_02, 자치구 != "합계" & 동 != "합계" & 동 != "소계")[-1,]

# 컴마, 하이픈 제거
for(j in 1:nrow(bw_02)){

    bw_02[j,4:ncol(bw_02)] <- str_replace_all(bw_02[j,4:ncol(bw_02)], ",", "")
    bw_02[j,4:ncol(bw_02)] <- str_replace_all(bw_02[j,4:ncol(bw_02)], "-", '0')
}

# 정수형 데이터로 변경
for(i in 4:ncol(bw_02)){
    
    bw_02[,i] <- as.integer(bw_02[,i])
}


# 열이름 변경
colnames(bw_02) <- c("년도", "자치구명", "동", "사업체수", "총종사자수", "총여성종사자수", colnames(bw_02[,7:ncol(bw_02)]) )

# 행이름 초기화
rownames(bw_02) <- c()

bw_02 <- subset(bw_02, 동 != '둔촌1동')



# .1 -> (여성합)

colnames(bw_02) <- str_replace_all(colnames(bw_02), ".1", "(여성)")
head(bw_02)

년도,자치구명,동,사업체수,총종사자수,총여성종사자수,농업.임업.및.어업,농업.임업.및.어업(여성),광업,광업(여성),...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정(여성),교육.서비스업,교육.서비스업(여성),보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업(여성),예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업(여성),협회.및.단체.수리.및.기타.개인.서비스업,협회.및.단체.수리.및.기타.개인.서비스업(여성)
2019,종로구,사직동,3574,47522,22368,12,4,2,1,...,6088,2369,786,474,1096,838,1653,794,906,475
2019,종로구,삼청동,732,4601,2330,0,0,0,0,...,1291,335,297,158,30,20,185,128,194,121
2019,종로구,부암동,599,3454,1690,0,0,1,0,...,60,13,1190,558,183,154,49,29,325,149
2019,종로구,평창동,761,3447,1929,0,0,4,1,...,90,31,425,281,480,392,185,100,266,150
2019,종로구,무악동,647,1644,772,0,0,0,0,...,12,6,177,134,486,384,11,7,90,51
2019,종로구,교남동,413,4581,3100,0,0,0,0,...,37,16,176,77,3139,2397,44,27,170,83


In [4]:
# 사업체, 종사자수, 총여성종사자수  만 추출
bw_03 <- bw_02[,1:6]

head(bw_03)
tail(bw_03)

# 동 갯수 확인
length(unique(bw_03$동))

# 자치구 개수 확인
length(unique(bw_03$자치구명))

table(bw_03$자치구명)

subset(bw_03, 자치구명 =="자치구")

ncol(bw_02)

년도,자치구명,동,사업체수,총종사자수,총여성종사자수
2019,종로구,사직동,3574,47522,22368
2019,종로구,삼청동,732,4601,2330
2019,종로구,부암동,599,3454,1690
2019,종로구,평창동,761,3447,1929
2019,종로구,무악동,647,1644,772
2019,종로구,교남동,413,4581,3100


,년도,자치구명,동,사업체수,총종사자수,총여성종사자수
418,2019,강동구,성내2동,1836,6770,3517
419,2019,강동구,성내3동,2439,9117,4130
421,2019,강동구,둔촌2동,1553,9463,4787
422,2019,강동구,암사1동,1918,5646,2869
423,2019,강동구,천호2동,3439,14208,7680
424,2019,강동구,길동,5037,21237,10139


[1] 422

[1] 25


  강남구   강동구   강북구   강서구   관악구   광진구   구로구   금천구 
      22       17       13       20       21       15       15       10 
  노원구   도봉구 동대문구   동작구   마포구 서대문구   서초구   성동구 
      19       14       14       15       16       14       18       17 
  성북구   송파구   양천구 영등포구   용산구   은평구   자치구   종로구 
      20       27       18       18       16       16        0       17 
    중구   중랑구     합계 
      15       16        0 

년도,자치구명,동,사업체수,총종사자수,총여성종사자수


[1] 44

In [5]:
# 자치구별 사업체 총종사자수 
aggregate(cbind(사업체수, 총종사자수, 총여성종사자수) ~ 자치구명, bw_02, sum)

table(bw_03$동)

자치구명,사업체수,총종사자수,총여성종사자수
강남구,71027,698840,310956
강동구,29063,141487,66982
강북구,19014,73904,39770
강서구,39458,257494,115285
관악구,25505,115329,55645
광진구,24445,126305,58406
구로구,38756,225668,96946
금천구,33814,242686,92097
노원구,26618,116684,61691
도봉구,18628,72939,35341



        가락1동         가락2동        가락본동        가리봉동          가산동 
              1               1               1               1               1 
        가양1동         가양2동         가양3동          가회동         갈현1동 
              1               1               1               1               1 
        갈현2동          강일동         개봉1동         개봉2동         개봉3동 
              1               1               1               1               1 
        개포1동         개포2동         개포4동         거여1동         거여2동 
              1               1               1               1               1 
        고덕1동         고덕2동         고척1동         고척2동          공덕동 
              1               1               1               1               1 
        공릉1동         공릉2동          공항동          광장동          광희동 
              1               1               1               1               1 
         교남동         구로1동         구로2동         구로3동         구로4동 
              1               1               1      

In [6]:
length(bw_03$동)

[1] 423

In [7]:
library(dplyr)

arrange(bw_02, desc(총종사자수))


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



년도,자치구명,동,사업체수,총종사자수,총여성종사자수,농업.임업.및.어업,농업.임업.및.어업(여성),광업,광업(여성),...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정(여성),교육.서비스업,교육.서비스업(여성),보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업(여성),예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업(여성),협회.및.단체.수리.및.기타.개인.서비스업,협회.및.단체.수리.및.기타.개인.서비스업(여성)
2019,금천구,가산동,16661,170886,60313,0,0,0,0,...,135,52,2190,1200,1091,771,454,214,1943,762
2019,영등포구,여의도동,8132,160633,60484,0,0,0,0,...,5196,1022,1775,1164,3860,3054,1363,617,4283,1719
2019,강남구,역삼1동,15451,155883,65738,3,1,0,0,...,775,400,5035,2801,6429,5062,1478,659,4269,2311
2019,중구,명동,6965,110390,50752,0,0,0,0,...,14632,6727,1038,578,2197,1649,480,246,2495,1391
2019,종로구,종로1.2.3.4가동,15472,108747,43034,0,0,3,1,...,4944,1578,3225,2064,1645,1169,1329,613,2835,1114
2019,구로구,구로3동,7883,92379,35909,20,4,1,0,...,328,163,2648,1540,1130,912,556,257,1588,631
2019,서초구,서초3동,7725,81064,32105,0,0,0,0,...,6771,2823,2676,1647,1509,1107,2340,1156,2343,859
2019,강남구,대치2동,3897,64496,25951,8,2,1,0,...,636,152,3800,2140,964,707,465,221,1006,479
2019,중구,소공동,3849,64144,27464,0,0,0,0,...,364,125,853,509,772,584,444,200,1472,593
2019,강남구,삼성1동,4079,63573,26578,0,0,0,0,...,266,50,696,435,1531,1173,1468,707,1264,676


In [8]:
head(bw_02)

년도,자치구명,동,사업체수,총종사자수,총여성종사자수,농업.임업.및.어업,농업.임업.및.어업(여성),광업,광업(여성),...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정(여성),교육.서비스업,교육.서비스업(여성),보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업(여성),예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업(여성),협회.및.단체.수리.및.기타.개인.서비스업,협회.및.단체.수리.및.기타.개인.서비스업(여성)
2019,종로구,사직동,3574,47522,22368,12,4,2,1,...,6088,2369,786,474,1096,838,1653,794,906,475
2019,종로구,삼청동,732,4601,2330,0,0,0,0,...,1291,335,297,158,30,20,185,128,194,121
2019,종로구,부암동,599,3454,1690,0,0,1,0,...,60,13,1190,558,183,154,49,29,325,149
2019,종로구,평창동,761,3447,1929,0,0,4,1,...,90,31,425,281,480,392,185,100,266,150
2019,종로구,무악동,647,1644,772,0,0,0,0,...,12,6,177,134,486,384,11,7,90,51
2019,종로구,교남동,413,4581,3100,0,0,0,0,...,37,16,176,77,3139,2397,44,27,170,83


In [9]:
# 15년도

bw15 <- read.table("../Data_SemiProject/2015_business_worker.txt",
           fileEncoding ="utf-8", sep="\t", header=T)

bw15_02 <- bw15


# factor -> character 변경
for(i in 4:ncol(bw15_02)){
    
    bw15_02[,i] <- as.character(bw15_02[,i])
}
str(bw15_02)

# 동별 사업체, 총 종사자 수 , 업종별 종사자 수 데이터
# 업종별 컬럼 앞에 .1 -> 해당 업종 여성 종사자 수

bw15_02 <- subset(bw15_02, 자치구 != "합계" & 동 != "합계" & 동 != "소계")[-1,]

# 컴마, 하이픈 제거
for(j in 1:nrow(bw15_02)){

    bw15_02[j,4:ncol(bw15_02)] <- str_replace_all(bw15_02[j,4:ncol(bw15_02)], ",", "")
    bw15_02[j,4:ncol(bw15_02)] <- str_replace_all(bw15_02[j,4:ncol(bw15_02)], "-", '0')
}

# 정수형 데이터로 변경
for(i in 4:ncol(bw15_02)){
    
    bw15_02[,i] <- as.integer(bw15_02[,i])
}


# 열이름 변경
colnames(bw15_02) <- c("년도", "자치구명", "동", "사업체수", "총종사자수", "총여성종사자수", colnames(bw15_02[,7:ncol(bw15_02)]) )

# 행이름 초기화
rownames(bw15_02) <- c()

bw15_02 <- subset(bw15_02, 동 != '둔촌1동')



# .1 -> (여성합)

colnames(bw15_02) <- str_replace_all(colnames(bw15_02), ".1", "(여성)")
head(bw15_02)

'data.frame':	451 obs. of  44 variables:
 $ 기간                                       : Factor w/ 2 levels "2015","기간": 2 1 1 1 1 1 1 1 1 1 ...
 $ 자치구                                     : Factor w/ 27 levels "강남구","강동구",..: 23 27 24 24 24 24 24 24 24 24 ...
 $ 동                                         : Factor w/ 426 levels "가락1동","가락2동",..: 89 400 211 163 171 151 392 118 31 9 ...
 $ 합계                                       : chr  "사업체수" "820,658" "40,871" "3,551" ...
 $ 합계.1                                     : chr  "총종사자수" "5,108,828" "269,106" "51,425" ...
 $ 합계.2                                     : chr  "여성종사자수" "2,221,109" "116,340" "21,644" ...
 $ 농업.임업.및.어업                          : chr  "소계" "491" "16" "16" ...
 $ 농업.임업.및.어업.1                        : chr  "여성종사자수" "90" "3" "3" ...
 $ 광업                                       : chr  "소계" "69" "7" "-" ...
 $ 광업.1                                     : chr  "여성종사자수" "12" "2" "-" ...
 $ 제조업                                     : 

년도,자치구명,동,사업체수,총종사자수,총여성종사자수,농업.임업.및.어업,농업.임업.및.어업(여성),광업,광업(여성),...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정(여성),교육.서비스업,교육.서비스업(여성),보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업(여성),예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업(여성),협회.및.단체.수리..및.기타.개인.서비스업,협회.및.단체.수리..및.기타.개인.서비스업(여성)
2015,종로구,사직동,3551,51425,21644,16,3,0,0,...,7691,2345,774,435,830,642,1541,757,1159,544
2015,종로구,삼청동,738,4736,2362,0,0,0,0,...,1164,273,357,184,34,24,158,111,197,128
2015,종로구,부암동,540,3635,1921,0,0,5,1,...,61,11,1496,795,90,76,60,37,450,203
2015,종로구,평창동,665,3059,1621,0,0,0,0,...,79,22,294,179,374,302,141,67,241,144
2015,종로구,무악동,581,1510,706,0,0,0,0,...,14,9,176,114,365,291,16,9,94,50
2015,종로구,교남동,320,3158,1816,0,0,0,0,...,34,14,200,62,1802,1285,26,14,159,67


In [10]:
# 16년도

bw16 <- read.table("../Data_SemiProject/2016_business_worker.txt",
           fileEncoding ="utf-8", sep="\t", header=T)

bw16_02 <- bw16


# factor -> character 변경
for(i in 4:ncol(bw16_02)){
    
    bw16_02[,i] <- as.character(bw16_02[,i])
}
str(bw16_02)

# 동별 사업체, 총 종사자 수 , 업종별 종사자 수 데이터
# 업종별 컬럼 앞에 .1 -> 해당 업종 여성 종사자 수

bw16_02 <- subset(bw16_02, 자치구 != "합계" & 동 != "합계" & 동 != "소계")[-1,]

# 컴마, 하이픈 제거
for(j in 1:nrow(bw16_02)){

    bw16_02[j,4:ncol(bw16_02)] <- str_replace_all(bw16_02[j,4:ncol(bw16_02)], ",", "")
    bw16_02[j,4:ncol(bw16_02)] <- str_replace_all(bw16_02[j,4:ncol(bw16_02)], "-", '0')
}

# 정수형 데이터로 변경
for(i in 4:ncol(bw16_02)){
    
    bw16_02[,i] <- as.integer(bw16_02[,i])
}


# 열이름 변경
colnames(bw16_02) <- c("년도", "자치구명", "동", "사업체수", "총종사자수", "총여성종사자수", colnames(bw16_02[,7:ncol(bw16_02)]) )

# 행이름 초기화
rownames(bw16_02) <- c()

bw16_02 <- subset(bw16_02, 동 != '둔촌1동')

# .1 -> (여성합)

colnames(bw16_02) <- str_replace_all(colnames(bw16_02), ".1", "(여성)")
head(bw16_02)

'data.frame':	451 obs. of  44 variables:
 $ 기간                                       : Factor w/ 2 levels "2016","기간": 2 1 1 1 1 1 1 1 1 1 ...
 $ 자치구                                     : Factor w/ 27 levels "강남구","강동구",..: 23 27 24 24 24 24 24 24 24 24 ...
 $ 동                                         : Factor w/ 426 levels "가락1동","가락2동",..: 89 400 211 163 171 151 392 118 31 9 ...
 $ 합계                                       : chr  "사업체수" "820,156" "40,664" "3,489" ...
 $ 합계.1                                     : chr  "총종사자수" "5,079,451" "268,309" "53,850" ...
 $ 합계.2                                     : chr  "여성종사자수" "2,232,049" "113,975" "22,208" ...
 $ 농업.임업.및.어업                          : chr  "소계" "463" "36" "18" ...
 $ 농업.임업.및.어업.1                        : chr  "여성종사자수" "77" "4" "4" ...
 $ 광업                                       : chr  "소계" "54" "5" "-" ...
 $ 광업.1                                     : chr  "여성종사자수" "6" "1" "-" ...
 $ 제조업                                     : c

년도,자치구명,동,사업체수,총종사자수,총여성종사자수,농업.임업.및.어업,농업.임업.및.어업(여성),광업,광업(여성),...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정(여성),교육.서비스업,교육.서비스업(여성),보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업(여성),예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업(여성),협회.및.단체.수리..및.기타.개인.서비스업,협회.및.단체.수리..및.기타.개인.서비스업(여성)
2016,종로구,사직동,3489,53850,22208,18,4,0,0,...,7165,2815,890,559,738,577,1514,765,1164,524
2016,종로구,삼청동,751,4823,2475,0,0,0,0,...,1199,282,354,188,29,22,163,112,253,161
2016,종로구,부암동,575,3690,1794,0,0,5,1,...,60,13,1530,702,90,72,80,48,476,222
2016,종로구,평창동,708,2990,1585,0,0,0,0,...,85,29,236,146,366,299,163,74,265,154
2016,종로구,무악동,573,1472,716,0,0,0,0,...,13,9,169,128,400,315,16,7,123,70
2016,종로구,교남동,302,3612,2108,0,0,0,0,...,42,11,150,54,2304,1581,23,10,182,86


In [11]:
# 17년도

bw17 <- read.table("../Data_SemiProject/2017_business_worker.txt",
           fileEncoding ="utf-8", sep="\t", header=T)

bw17_02 <- bw17


# factor -> character 변경
for(i in 4:ncol(bw17_02)){
    
    bw17_02[,i] <- as.character(bw17_02[,i])
}
str(bw17_02)

# 동별 사업체, 총 종사자 수 , 업종별 종사자 수 데이터
# 업종별 컬럼 앞에 .1 -> 해당 업종 여성 종사자 수

bw17_02 <- subset(bw17_02, 자치구 != "합계" & 동 != "합계" & 동 != "소계")[-1,]

# 컴마, 하이픈 제거
for(j in 1:nrow(bw17_02)){

    bw17_02[j,4:ncol(bw17_02)] <- str_replace_all(bw17_02[j,4:ncol(bw17_02)], ",", "")
    bw17_02[j,4:ncol(bw17_02)] <- str_replace_all(bw17_02[j,4:ncol(bw17_02)], "-", '0')
}

# 정수형 데이터로 변경
for(i in 4:ncol(bw17_02)){
    
    bw17_02[,i] <- as.integer(bw17_02[,i])
}


# 열이름 변경
colnames(bw17_02) <- c("년도", "자치구명", "동", "사업체수", "총종사자수", "총여성종사자수", colnames(bw17_02[,7:ncol(bw17_02)]) )

# 행이름 초기화
rownames(bw17_02) <- c()

bw17_02 <- subset(bw17_02, 동 != '둔촌1동')



# .1 -> (여성합)

colnames(bw17_02) <- str_replace_all(colnames(bw17_02), ".1", "(여성)")
head(bw17_02)

'data.frame':	451 obs. of  44 variables:
 $ 기간                                      : Factor w/ 2 levels "2017","기간": 2 1 1 1 1 1 1 1 1 1 ...
 $ 자치구                                    : Factor w/ 27 levels "강남구","강동구",..: 23 27 24 24 24 24 24 24 24 24 ...
 $ 동                                        : Factor w/ 426 levels "가락1동","가락2동",..: 89 400 211 163 171 151 392 118 31 9 ...
 $ 합계                                      : chr  "사업체수" "822,863" "40,490" "3,544" ...
 $ 합계.1                                    : chr  "총종사자수" "5,119,913" "268,702" "52,489" ...
 $ 합계.2                                    : chr  "여성종사자수" "2,271,763" "114,602" "21,460" ...
 $ 농업.임업.및.어업                         : chr  "계" "469" "30" "16" ...
 $ 농업.임업.및.어업.1                       : chr  "여성종사자수" "88" "10" "4" ...
 $ 광업                                      : chr  "계" "66" "10" "-" ...
 $ 광업.1                                    : chr  "여성종사자수" "8" "2" "-" ...
 $ 제조업                                    : chr  "계" "27

년도,자치구명,동,사업체수,총종사자수,총여성종사자수,농업.임업.및.어업,농업.임업.및.어업(여성),광업,광업(여성),...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정(여성),교육.서비스업,교육.서비스업(여성),보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업(여성),예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업(여성),협회.및.단체.수리.및.기타.개인.서비스업,협회.및.단체.수리.및.기타.개인.서비스업(여성)
2017,종로구,사직동,3544,52489,21460,16,4,0,0,...,7667,2548,898,549,764,588,1546,708,920,464
2017,종로구,삼청동,753,4507,2282,0,0,0,0,...,1229,279,279,148,28,22,172,116,216,132
2017,종로구,부암동,567,4147,2010,0,0,5,1,...,58,9,1543,832,153,129,73,46,806,227
2017,종로구,평창동,739,3547,1868,0,0,0,0,...,77,29,661,377,385,315,167,74,303,161
2017,종로구,무악동,531,1457,728,0,0,0,0,...,14,9,174,126,469,378,14,4,95,46
2017,종로구,교남동,347,3816,2263,0,0,0,0,...,37,13,165,69,2490,1722,31,12,159,68


In [12]:
# 18년도

bw18 <- read.table("../Data_SemiProject/2018_business_worker.txt",
           fileEncoding ="utf-8", sep="\t", header=T)

bw18_02 <- bw18


# factor -> character 변경
for(i in 4:ncol(bw18_02)){
    
    bw18_02[,i] <- as.character(bw18_02[,i])
}
str(bw18_02)

# 동별 사업체, 총 종사자 수 , 업종별 종사자 수 데이터
# 업종별 컬럼 앞에 .1 -> 해당 업종 여성 종사자 수

bw18_02 <- subset(bw18_02, 자치구 != "합계" & 동 != "합계" & 동 != "소계")[-1,]

# 컴마, 하이픈 제거
for(j in 1:nrow(bw18_02)){

    bw18_02[j,4:ncol(bw18_02)] <- str_replace_all(bw18_02[j,4:ncol(bw18_02)], ",", "")
    bw18_02[j,4:ncol(bw18_02)] <- str_replace_all(bw18_02[j,4:ncol(bw18_02)], "-", '0')
}

# 정수형 데이터로 변경
for(i in 4:ncol(bw18_02)){
    
    bw18_02[,i] <- as.integer(bw18_02[,i])
}


# 열이름 변경
colnames(bw18_02) <- c("년도", "자치구명", "동", "사업체수", "총종사자수", "총여성종사자수", colnames(bw18_02[,7:ncol(bw18_02)]) )

# 행이름 초기화
rownames(bw18_02) <- c()

bw18_02 <- subset(bw18_02, 동 != '둔촌1동')



# .1 -> (여성합)

colnames(bw18_02) <- str_replace_all(colnames(bw18_02), ".1", "(여성)")
head(bw18_02)

'data.frame':	451 obs. of  44 variables:
 $ 기간                                      : Factor w/ 2 levels "2018","기간": 2 1 1 1 1 1 1 1 1 1 ...
 $ 자치구                                    : Factor w/ 27 levels "강남구","강동구",..: 23 27 24 24 24 24 24 24 24 24 ...
 $ 동                                        : Factor w/ 426 levels "가락1동","가락2동",..: 89 400 211 163 171 151 392 118 31 9 ...
 $ 합계                                      : chr  "사업체수" "823,385" "39,952" "3,541" ...
 $ 합계.1                                    : chr  "총종사자수" "5,210,936" "265,017" "49,536" ...
 $ 합계.2                                    : chr  "여성종사자수" "2,332,709" "112,300" "20,878" ...
 $ 농업.임업.및.어업                         : chr  "계" "462" "17" "14" ...
 $ 농업.임업.및.어업.1                       : chr  "여성종사자수" "78" "4" "3" ...
 $ 광업                                      : chr  "계" "81" "10" "-" ...
 $ 광업.1                                    : chr  "여성종사자수" "19" "2" "-" ...
 $ 제조업                                    : chr  "계" "26

년도,자치구명,동,사업체수,총종사자수,총여성종사자수,농업.임업.및.어업,농업.임업.및.어업(여성),광업,광업(여성),...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정(여성),교육.서비스업,교육.서비스업(여성),보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업(여성),예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업(여성),협회.및.단체.수리.및.기타.개인.서비스업,협회.및.단체.수리.및.기타.개인.서비스업(여성)
2018,종로구,사직동,3541,49536,20878,14,3,0,0,...,6312,2795,967,614,870,673,1544,693,843,464
2018,종로구,삼청동,712,4577,2384,0,0,0,0,...,1204,292,260,144,30,21,175,124,209,125
2018,종로구,부암동,565,3609,1903,0,0,1,0,...,59,9,1424,742,148,137,40,18,362,196
2018,종로구,평창동,751,3565,1895,0,0,5,1,...,85,26,618,364,466,367,186,91,262,137
2018,종로구,무악동,581,1598,795,0,0,0,0,...,16,11,185,140,482,382,12,2,102,60
2018,종로구,교남동,393,4008,1522,0,0,0,0,...,40,16,176,82,2499,828,44,17,181,83


In [13]:
bw19_02 <- bw_02

# 열 이름 선언 뒤 초기화
colnames(bw15_02) <- colnames(bw_02)
colnames(bw16_02) <- colnames(bw_02)
colnames(bw17_02) <- colnames(bw_02)
colnames(bw18_02) <- colnames(bw_02)
colnames(bw19_02) <- colnames(bw_02)

# 15~19 합치기
business_worker <- rbind(bw15_02, bw16_02, bw17_02, bw18_02, bw19_02)

# 년도를 오름차순후 년도 기준에서 자치구명 오름차순, 그 기준에서 동 오름차순 정렬
business_worker <- arrange(business_worker, business_worker$년도, business_worker$자치구명, business_worker$동)

# 사업체, 종사자수 최종
business_worker

length(unique(business_worker$년도))
length(unique(business_worker$자치구명))
length(unique(business_worker$동))




년도,자치구명,동,사업체수,총종사자수,총여성종사자수,농업.임업.및.어업,농업.임업.및.어업(여성),광업,광업(여성),...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정(여성),교육.서비스업,교육.서비스업(여성),보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업(여성),예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업(여성),협회.및.단체.수리.및.기타.개인.서비스업,협회.및.단체.수리.및.기타.개인.서비스업(여성)
2015,강남구,개포1동,490,1694,582,0,0,0,0,...,15,8,231,185,65,58,4,1,100,56
2015,강남구,개포2동,902,3466,1988,0,0,0,0,...,15,10,1264,913,189,152,100,53,270,170
2015,강남구,개포4동,2403,12402,4077,0,0,0,0,...,78,16,477,319,649,517,236,106,584,209
2015,강남구,논현1동,5770,43779,20314,0,0,0,0,...,68,18,711,454,2464,1926,644,269,1440,845
2015,강남구,논현2동,5731,65282,27341,0,0,5,0,...,649,260,2007,1179,1767,1342,701,236,2087,1087
2015,강남구,대치1동,1013,7103,3974,0,0,0,0,...,148,52,2721,1544,670,531,141,70,364,211
2015,강남구,대치2동,3579,51945,21308,0,0,5,1,...,138,85,2954,1601,900,618,438,214,867,424
2015,강남구,대치4동,3756,29049,12176,0,0,0,0,...,99,77,2767,1452,1039,797,432,183,1040,552
2015,강남구,도곡1동,1656,19542,8015,0,0,0,0,...,0,0,748,465,2837,2131,298,95,483,196
2015,강남구,도곡2동,1970,22083,8674,0,0,0,0,...,18,6,978,673,347,289,253,103,435,222


[1] 5

[1] 25

[1] 423

In [14]:
write.table(business_worker ,"../Data_SemiProject/2015-2019_business_worker.txt")

In [15]:
head(business_worker)

년도,자치구명,동,사업체수,총종사자수,총여성종사자수,농업.임업.및.어업,농업.임업.및.어업(여성),광업,광업(여성),...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정(여성),교육.서비스업,교육.서비스업(여성),보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업(여성),예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업(여성),협회.및.단체.수리.및.기타.개인.서비스업,협회.및.단체.수리.및.기타.개인.서비스업(여성)
2015,강남구,개포1동,490,1694,582,0,0,0,0,...,15,8,231,185,65,58,4,1,100,56
2015,강남구,개포2동,902,3466,1988,0,0,0,0,...,15,10,1264,913,189,152,100,53,270,170
2015,강남구,개포4동,2403,12402,4077,0,0,0,0,...,78,16,477,319,649,517,236,106,584,209
2015,강남구,논현1동,5770,43779,20314,0,0,0,0,...,68,18,711,454,2464,1926,644,269,1440,845
2015,강남구,논현2동,5731,65282,27341,0,0,5,0,...,649,260,2007,1179,1767,1342,701,236,2087,1087
2015,강남구,대치1동,1013,7103,3974,0,0,0,0,...,148,52,2721,1544,670,531,141,70,364,211


In [16]:
subset(business_worker, 동 == "신사동")


,년도,자치구명,동,사업체수,총종사자수,총여성종사자수,농업.임업.및.어업,농업.임업.및.어업(여성),광업,광업(여성),...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정(여성),교육.서비스업,교육.서비스업(여성),보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업(여성),예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업(여성),협회.및.단체.수리.및.기타.개인.서비스업,협회.및.단체.수리.및.기타.개인.서비스업(여성)
15,2015,강남구,신사동,4958,29398,15240,0,0,0,0,...,63,20,1378,856,3091,2482,431,185,1146,728
85,2015,관악구,신사동,1428,4470,2245,0,0,0,0,...,42,11,369,298,366,303,100,35,327,160
438,2016,강남구,신사동,4503,27741,14505,0,0,0,0,...,46,12,1254,800,3216,2565,375,178,966,630
508,2016,관악구,신사동,1422,4712,2184,0,0,0,0,...,54,16,338,262,346,293,118,43,324,159
861,2017,강남구,신사동,4365,26783,14346,0,0,0,0,...,54,15,1296,839,3359,2717,429,210,1017,670
931,2017,관악구,신사동,1384,4721,2223,0,0,0,0,...,53,17,326,248,438,371,109,44,326,157
1284,2018,강남구,신사동,4223,26902,14238,0,0,0,0,...,61,20,1311,865,3406,2752,550,247,1000,649
1354,2018,관악구,신사동,1388,4696,2233,0,0,0,0,...,54,17,373,277,467,400,125,48,319,154
1707,2019,강남구,신사동,4140,26210,14106,0,0,0,0,...,60,12,1266,851,3305,2661,474,221,972,649
1777,2019,관악구,신사동,1359,4356,2094,0,0,0,0,...,78,14,308,222,450,390,149,58,339,168


In [18]:
for(i in 2:3){
    
    business_worker[,i] <- as.character(business_worker[,i])
}




'data.frame':	2115 obs. of  44 variables:
 $ 년도                                          : Factor w/ 6 levels "2015","기간",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ 자치구명                                      : chr  "강남구" "강남구" "강남구" "강남구" ...
 $ 동                                            : chr  "개포1동" "개포2동" "개포4동" "논현1동" ...
 $ 사업체수                                      : int  490 902 2403 5770 5731 1013 3579 3756 1656 1970 ...
 $ 총종사자수                                    : int  1694 3466 12402 43779 65282 7103 51945 29049 19542 22083 ...
 $ 총여성종사자수                                : int  582 1988 4077 20314 27341 3974 21308 12176 8015 8674 ...
 $ 농업.임업.및.어업                             : int  0 0 0 0 0 0 0 0 0 0 ...
 $ 농업.임업.및.어업(여성)                       : int  0 0 0 0 0 0 0 0 0 0 ...
 $ 광업                                          : int  0 0 0 0 5 0 5 0 0 0 ...
 $ 광업(여성)                                    : int  0 0 0 0 0 0 1 0 0 0 ...
 $ 제조업                                        : int  11 26 299 

In [19]:
# 신사동_강남, 신사동_관악 구분자

for(i in 1:nrow(business_worker)){
    
    if(business_worker$자치구명[i] == '강남구' & business_worker$동[i] == '신사동'){
        
        business_worker$동[i] <- '신사동_강남'
    }else if(business_worker$자치구명[i] == '관악구' & business_worker$동[i] == '신사동'){
        
        business_worker$동[i] <- '신사동_관악'
    }
    
}

In [23]:
subset(business_worker, 동 == '신사동_관악')

,년도,자치구명,동,사업체수,총종사자수,총여성종사자수,농업.임업.및.어업,농업.임업.및.어업(여성),광업,광업(여성),...,공공행정.국방.및.사회보장.행정,공공행정.국방.및.사회보장.행정(여성),교육.서비스업,교육.서비스업(여성),보건업.및.사회복지.서비스업,보건업.및.사회복지.서비스업(여성),예술.스포츠.및.여가관련.서비스업,예술.스포츠.및.여가관련.서비스업(여성),협회.및.단체.수리.및.기타.개인.서비스업,협회.및.단체.수리.및.기타.개인.서비스업(여성)
85,2015,관악구,신사동_관악,1428,4470,2245,0,0,0,0,...,42,11,369,298,366,303,100,35,327,160
508,2016,관악구,신사동_관악,1422,4712,2184,0,0,0,0,...,54,16,338,262,346,293,118,43,324,159
931,2017,관악구,신사동_관악,1384,4721,2223,0,0,0,0,...,53,17,326,248,438,371,109,44,326,157
1354,2018,관악구,신사동_관악,1388,4696,2233,0,0,0,0,...,54,17,373,277,467,400,125,48,319,154
1777,2019,관악구,신사동_관악,1359,4356,2094,0,0,0,0,...,78,14,308,222,450,390,149,58,339,168


In [24]:
# 신사동_강남, 신사동_관악 으로 구분자

write.table(business_worker ,"../Data_SemiProject/2015-2019_business_worker_02.txt")